In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from util import ManualMFCC, SpeechDataset, SpeechRecognizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# 初始化模型、优化器和损失函数
model = SpeechRecognizer().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()  # 改为交叉熵损失

In [3]:
# 初始化MFCC提取器
mfcc_extractor = ManualMFCC(sample_rate=8000)

# 创建数据集和数据加载器
VOCAB = {"00": "数字", "01": "语音", "02": "语言", "03": "处理", "04": "中国", "05": "忠告", "06": "北京", "07": "背景", "08": "上海", "09": "商行", "10": "Speech", "11": "Speaker", "12": "Signal", "13": "Sequence", "14": "Processing", "15": "Print", "16": "Project", "17": "File", "18": "Open", "19": "Close"}
dataset = SpeechDataset("", VOCAB, mfcc_extractor)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

训练集: 2400 个音频文件


In [ ]:
num_epochs = 100
scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-5)

for epoch in range(num_epochs):
    model.train()
    total_loss, correct, total = 0.0, 0, 0

    for batch_idx, (mfcc, labels) in enumerate(dataloader):
        # 调整维度为 (batch, time, features)
        mfcc = mfcc.permute(0, 2, 1).to(device)
        labels = labels.squeeze().to(device)  # 从(batch,1)变为(batch,)
        
        # 前向传播
        outputs = model(mfcc)
        
        # 计算损失
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 统计
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # 更新学习率
    scheduler.step()
    
    epoch_loss = total_loss / len(dataloader)
    epoch_acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

torch.save(model.state_dict(), "model.pth")

Epoch [1/1], Loss: 0.0016, Accuracy: 99.96%
